# Using Examples (Few‑Shot Prompting)
**Modèle** : *Llama-3.2-1B (Instruct)* via **Ollama**  
**Objectif** : Améliorer la qualité et la stabilité des réponses en guidant le modèle avec des **exemples** (few‑shot), tout en contrôlant le style et le format.

## Pourquoi le few‑shot ?
- **Guidage** : démontre *comment* répondre (forme, ton, granularité) plutôt que *quoi* répondre seulement.
- **Généralisation** : le modèle imite le **pattern** fourni.
- **Stabilité** : réduit la variabilité vs zéro‑shot, surtout pour des formats stricts.

In [ ]:
# (Optionnel) Installer/mettre à jour le client Python Ollama
# !pip install -q --upgrade ollama

In [1]:
import ollama

def assert_ollama_ready():
    try:
        _ = ollama.list()
        print("✅ Ollama est accessible en local.")
    except Exception as e:
        raise SystemExit(
            "❌ Impossible de contacter Ollama. Démarrez le serveur (ex: 'ollama serve').\n"
            f"Détails: {e}"
        )

assert_ollama_ready()

try:
    print("⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...")
    ollama.pull('llama3.2:1b')
    print("✅ Modèle prêt.")
except Exception as e:
    print("ℹ️ Vérifiez le tag (ex: 'llama3.2:1b').")
    raise

✅ Ollama est accessible en local.
⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...
✅ Modèle prêt.


## Fonctions utilitaires
- `chat_zeroshot(user)` : baseline sans exemple.  
- `chat_fewshot(system, shots, user)` : ajoute des **exemples** (rôle `assistant` ou paires user/assistant).

In [3]:
from typing import List, Tuple, Optional

def chat_zeroshot(user_prompt: str, temperature: float = 0.2) -> str:
    r = ollama.chat(model="llama3.2:1b", messages=[{"role":"user","content": user_prompt}], options={"temperature": temperature})
    return r["message"]["content"]

def chat_fewshot(system_prompt: str, shots: List[Tuple[str, str]], user_prompt: str, temperature: float = 0.2) -> str:
    messages = []
    if system_prompt:
        messages.append({"role":"system","content": system_prompt})
   
    for u,a in shots:
        messages.append({"role":"user","content": u})
        messages.append({"role":"assistant","content": a})
    messages.append({"role":"user","content": user_prompt})
    r = ollama.chat(model="llama3.2:1b", messages=messages, options={"temperature": temperature})
    return r["message"]["content"]

## 1) Zéro‑shot vs Few‑shot (format imposé)
Nous voulons un **résumé en 3 puces** avec **une phrase** par puce.

In [4]:
text = (
    "Les arbres de décision partitionnent l'espace des caractéristiques en régions homogènes. "
    "Ils sont lisibles, rapides à entraîner, mais peuvent sur‑apprendre sans élagage. "
    "On les utilise aussi en ensembles (forêts aléatoires, boosting) pour de meilleures performances."
)
user_q = f"Résume ce texte en 3 puces, une phrase par puce. Texte: {text}"
print("— Zéro‑shot —\n", chat_zeroshot(user_q))

system_rule = "Tu es un formateur concis. Sortie: exactement 3 puces, une phrase par puce."
shots = [
    ("Résume: 'Le k‑NN classe par proximité moyenne.'",
     "- Méthode paresseuse basée sur la proximité des exemples.\n- Performance sensible au choix de k et à la normalisation.\n- Utile quand la frontière est non linéaire et que les données sont locales."),
]
print("\n— Few‑shot (avec 1 exemple) —\n", chat_fewshot(system_rule, shots, user_q))

— Zéro‑shot —
 Voici un résumé du texte en 3 puces :

* Les arbres de décision sont des éléments clés qui partitionnent l'espace dans des régions homogènes.
* Ils sont faciles à comprendre et rapides à entraîner, mais peuvent sur-apprendre sans élagage.
* On les utilise souvent en ensemble (forêts aléatoires ou boosting) pour améliorer les performances.

— Few‑shot (avec 1 exemple) —
 - Les arbres de décision sont des structures d'apprentissage automatique.
- Ils partitionnent l'espace des caractéristiques en régions homogènes.
- On les utilise pour améliorer la performance des modèles de classification et d'analyse.


## 2) Style transfer par l’exemple
Nous voulons un style **humoristique** et **grand public**.

In [5]:
system_style = "Tu écris pour le grand public, ton léger et analogies simples. 3 phrases."
shots = [
    ("Explique le sur‑apprentissage.",
     "Imagine un élève qui mémorise chaque réponse sans comprendre: il cartonne en devoir maison mais panique à l'exam. "
     "Un modèle sur‑apprend pareil: super sur les données vues, nul sur le reste. "
     "La solution: plus de données, régularisation, et un peu d'humilité.")
]
user_q = "Explique les réseaux de neurones en 3 phrases."
print(chat_fewshot(system_style, shots, user_q, temperature=0.3))

Les réseaux de neurones sont des systèmes informatiques qui apprennent à reconnaitre et à comprendre les données en utilisant des connexions artificielles. Ils fonctionnent comme un réseau d'informateurs qui échangent des informations, ce qui leur permet de prendre des décisions ou d'apprendre de nouvelles choses. Les réseaux de neurones sont souvent utilisés dans les applications de traitement du langage naturel, la reconnaissance d'images et le traitement des données sensorielles.


## 3) Sorties structurées guidées par exemples
On exige un **JSON** avec clés `definition`, `use_cases` (3), `risk` (1 phrase).

In [6]:
system_json = "Tu es un générateur de JSON strict. Réponds uniquement par un objet JSON valide."
shots = [
    (
        "Donne un JSON pour 'Réseau bayésien' avec les mêmes clés.",
        '{"definition":"Modèle probabiliste à graphes orientés représentant des dépendances.",'
        '"use_cases":["diagnostic médical","détection d\'anomalies","aide à la décision"],'
        '"risk":"Sensibilité aux hypothèses d\'indépendance et aux données partielles."}'
    )
]
user_q = "Donne un JSON pour 'Transformers' avec les mêmes clés."
print(chat_fewshot(system_json, shots, user_q, temperature=0.1))

```json
{
  "name": "Transformers",
  "description": "Modèle de traitement automatique du langage à base d'apprentissage automatique et de réseaux de neurones.",
  "use_cases": [
    "Recherche d'entraînement des modèles de langage",
    "Analyse de texte",
    "Réduction de la taille des données"
  ],
  "risk": {
    "sensibilité": "Élevée",
    "hypothèses d'indépendance": "Faible",
    "données partielles": "Faible"
  }
}
```


## 4) Contre‑exemples (negative shots)
Fournir un exemple **à ne pas imiter** peut clarifier le style attendu.

In [12]:
system_neg = (
    "Tu es un rédacteur technique. Évite le jargon inutile. Sortie: 4 puces, phrases courtes."
)
shots = [
    ("Mauvais style: explication verbeuse et compliquée.",
     "(À éviter) Le café, décoction aromatique résultant de la percolation aqueuse à haute température..."),
    ("Bon style: 4 puces, phrases simples.",
     "- Boisson chaude.\n- Fait à partir de grains moulus.\n- Donne de l’énergie.\n- Trop en boire empêche de dormir."),
]

user_q = "Explique le café en 4 puces."
print(chat_fewshot(system_neg, shots, user_q, temperature=0.2))

Voici une explication du café en 4 puces :

*   Le café est un produit alimentaire résultant de la fermentation des graines moulues.
*   Il est généralement consommé chaud, souvent accompagné d'un petit déjeuner ou d'une pause après le travail.
*   Le café contient une grande quantité de caféine, qui peut avoir des effets stimulants sur l'organisme.


## 5) Nombre d’exemples & ordre
- Trop d’exemples peut **allonger** et **diluer** l’instruction.
- L’**ordre** des exemples influence souvent le style final (effet de récence).

In [14]:
system_rule = "Tu réponds en 3 puces, ton pédagogique."
shots1 = [
    ("Explique le gradient.", "- Direction d\'amélioration.\n- Calcule la pente.\n- Guide la mise à jour."),
]
shots3 = [
    ("Explique la régularisation.", "- Pénalise la complexité.\n- Réduit le sur‑apprentissage.\n- Améliore la généralisation."),
    ("Explique la validation croisée.", "- Sépare entraînement/validation.\n- Évalue la robustesse.\n- Aide au choix d\'hyperparamètres."),
    ("Explique le taux d\'apprentissage.", "- Contrôle la taille des pas.\n- Trop grand: diverge.\n- Trop petit: lent."),
]
user_q = "Explique le dropout."
print("— 1 shot —\n", chat_fewshot(system_rule, shots1, user_q))
print("\n— 3 shots —\n", chat_fewshot(system_rule, shots3, user_q))

— 1 shot —
 Le dropout est un concept clé dans l'apprentissage automatique et les réseaux de neurones artificiels (RNN). Il s'agit d'une technique utilisée pour supprimer certaines unités ou des lignes du réseau, généralement pour éviter les "pentes" ou les "déviations" non prévisibles.

En général, le dropout est appliqué à un réseau de neurones artificiels lorsqu'il est entraîné sur une grande quantité d'entraînement. Lorsque le réseau est entraîné, il peut apprendre à reconnaître des modèles dans les données, mais cela peut également conduire à la généralisation excessive et à l'apparition de "déviations" non prévisibles.

Le dropout permet de supprimer ces déviations en sélectionnant aléatoirement certaines unités du réseau pour ne pas les entraîner. Cela aide à :

- Améliorer la stabilité du modèle
- Réduire l'overfitting (apparition excessive d'un modèle sur un ensemble de données)
- Augmenter la généralisation

Le dropout est souvent utilisé dans les réseaux de neurones artifici

## 6) Pièges courants & bonnes pratiques
- **Sur‑contrainte** : n’écrasez pas la flexibilité si la tâche est ouverte.
- **Exemples biaisés** : diversifiez pour éviter une sortie trop uniforme.
- **Hallucination de format** : répétez les clés exactes dans les exemples.
- **Trop de shots** : coût/latence ↑ ; privilégiez 1–3 exemples pertinents.

## Template few‑shot réutilisable
System: « Tu es <persona>. Sortie: <format>. Contraintes: <N puces / M phrases / JSON>. »

Exemple user → assistant #1
- User: <entrée‑exemple>
- Assistant: <sortie‑exemple>

Exemple user → assistant #2 (optionnel)

User (nouvelle entrée): <votre question>

## À retenir (Phase 5)
- Le **few‑shot** montre des **patrons** (format, ton) à imiter.  
- 1–3 exemples **bien choisis** suffisent souvent.  
- Combinez avec les phases 2–4 : persona (system), séparation data/instructions, format strict.

